In [23]:
#The Boarded Window by Ambrose Bierce

"Sample code to use the IBM Watson Speech to Text API."
import json
from watson_developer_cloud import SpeechToTextV1

IBM_USERNAME = "apikey"
IBM_PASSWORD = 'Pass' #Put your API key here

stt = SpeechToTextV1(username=IBM_USERNAME, password=IBM_PASSWORD)
audio_file = open("new_story.wav", "rb") #audio file's name


with open('new_story.json', 'w') as fp:
    result = stt.recognize(audio_file, content_type="audio/wav",
                           continuous=True, timestamps=False,
                           max_alternatives=1).get_result()
    json.dump(result, fp, indent=2)

In [43]:
#Read the lines from the json file
with open('new_story.json') as file:
    distros_dict = json.load(file)
for item in distros_dict['results']:
    print(item['alternatives'][0]['transcript'] +'.')

our story today is called the boarded window it was written by Ambrose Bierce here is shop o'neill with the story .
in eighteen thirty only a few miles away from what is now the great city of Cincinnati Ohio they are huge and almost endless forest .
the area had %HESITATION few settlements established by people of the front two year many of them had already left the area for settlements further to the west but among those remaining was a man who had been one of the first people to arrive there .
he lived alone in a house of logs surrounded on all sides by the great forest .
he seemed a part of the darkness and silence of the forest for it no one had ever known him to smile or speak an unnecessary word .
his simple needs were supplied by selling or trading the skins of wild animals in the town .
his little log house had a single door .
directly opposite was a window .
the window was boarded up no one could remember a time when it was not .
and no one knew why it had been close .
I imagi

In [47]:
#Convert lines to a single string
movie=[]
for item in distros_dict['results']:
    movie.append(item['alternatives'][0]['transcript'])
script='. '.join(movie)
script

"our story today is called the boarded window it was written by Ambrose Bierce here is shop o'neill with the story . in eighteen thirty only a few miles away from what is now the great city of Cincinnati Ohio they are huge and almost endless forest . the area had %HESITATION few settlements established by people of the front two year many of them had already left the area for settlements further to the west but among those remaining was a man who had been one of the first people to arrive there . he lived alone in a house of logs surrounded on all sides by the great forest . he seemed a part of the darkness and silence of the forest for it no one had ever known him to smile or speak an unnecessary word . his simple needs were supplied by selling or trading the skins of wild animals in the town . his little log house had a single door . directly opposite was a window . the window was boarded up no one could remember a time when it was not . and no one knew why it had been close . I imag

In [48]:
#Preprocessing
import re

processed_movie = re.sub(r'\[[0-9]*\]', ' ', script)  
processed_movie = re.sub(r'\s+', ' ', script)

# Removing special characters and digits
formatted_movie = re.sub('[^a-zA-Z]', ' ', processed_movie )  
formatted_movie = re.sub(r'\s+', ' ', processed_movie)  

In [49]:
#Converting Text To Sentences
import nltk
sentence_list = nltk.sent_tokenize(processed_movie)  
sentence_list

["our story today is called the boarded window it was written by Ambrose Bierce here is shop o'neill with the story .",
 'in eighteen thirty only a few miles away from what is now the great city of Cincinnati Ohio they are huge and almost endless forest .',
 'the area had %HESITATION few settlements established by people of the front two year many of them had already left the area for settlements further to the west but among those remaining was a man who had been one of the first people to arrive there .',
 'he lived alone in a house of logs surrounded on all sides by the great forest .',
 'he seemed a part of the darkness and silence of the forest for it no one had ever known him to smile or speak an unnecessary word .',
 'his simple needs were supplied by selling or trading the skins of wild animals in the town .',
 'his little log house had a single door .',
 'directly opposite was a window .',
 'the window was boarded up no one could remember a time when it was not .',
 'and no on

In [52]:
#Find Weighted Frequency of Occurrence

stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}  
for word in nltk.word_tokenize(formatted_movie):  
    if word not in stopwords and word not in ".":
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
word_frequencies

{'story': 6,
 'today': 2,
 'called': 1,
 'boarded': 3,
 'window': 8,
 'written': 2,
 'Ambrose': 2,
 'Bierce': 2,
 'shop': 1,
 "o'neill": 2,
 'eighteen': 1,
 'thirty': 1,
 'miles': 2,
 'away': 3,
 'great': 3,
 'city': 1,
 'Cincinnati': 1,
 'Ohio': 1,
 'huge': 2,
 'almost': 1,
 'endless': 1,
 'forest': 6,
 'area': 3,
 '%': 1,
 'HESITATION': 1,
 'settlements': 2,
 'established': 1,
 'people': 4,
 'front': 1,
 'two': 2,
 'year': 2,
 'many': 4,
 'already': 1,
 'left': 3,
 'west': 1,
 'among': 1,
 'remaining': 1,
 'man': 6,
 'one': 10,
 'first': 1,
 'arrive': 1,
 'lived': 2,
 'alone': 2,
 'house': 3,
 'logs': 1,
 'surrounded': 1,
 'sides': 1,
 'seemed': 2,
 'part': 4,
 'darkness': 5,
 'silence': 2,
 'ever': 1,
 'known': 3,
 'smile': 1,
 'speak': 2,
 'unnecessary': 1,
 'word': 1,
 'simple': 1,
 'needs': 1,
 'supplied': 2,
 'selling': 1,
 'trading': 1,
 'skins': 1,
 'wild': 3,
 'animals': 3,
 'town': 1,
 'little': 3,
 'log': 1,
 'single': 1,
 'door': 1,
 'directly': 1,
 'opposite': 1,
 'could'

In [53]:
#divide the number of occurances of all the words by the frequency of the most occurring word
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
maximum_frequncy

11

In [54]:
#Calculating Sentence Scores

sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
sentence_scores

{"our story today is called the boarded window it was written by Ambrose Bierce here is shop o'neill with the story .": 2.818181818181818,
 'in eighteen thirty only a few miles away from what is now the great city of Cincinnati Ohio they are huge and almost endless forest .': 1.909090909090909,
 'he lived alone in a house of logs surrounded on all sides by the great forest .': 1.7272727272727273,
 'he seemed a part of the darkness and silence of the forest for it no one had ever known him to smile or speak an unnecessary word .': 3.454545454545454,
 'his simple needs were supplied by selling or trading the skins of wild animals in the town .': 1.2727272727272727,
 'his little log house had a single door .': 0.8181818181818182,
 'directly opposite was a window .': 0.9090909090909092,
 'the window was boarded up no one could remember a time when it was not .': 2.818181818181818,
 'and no one knew why it had been close .': 1.4545454545454546,
 'I imagine there are few people living today 

In [79]:
#Getting the Summary
import heapq  

summary_sentences = heapq.nlargest(8, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)  
print(summary) 

%HESITATION well novel I'm sure we'd all be fascinated to hear what you have to say . I will never lie and sure we can find a place for you in all right let's say something . well I must say I'd hoped for the best . doesn't matter how is gone sundown of people die every day . well done drink .


In [61]:
#Harry Potter
#Convert to .wav file

"""Sample code to use the IBM Watson Speech to Text API.
"""
import json
from watson_developer_cloud import SpeechToTextV1

IBM_USERNAME = "apikey"
IBM_PASSWORD = 'Pass'

stt = SpeechToTextV1(username=IBM_USERNAME, password=IBM_PASSWORD)
audio_file = open("harry.wav", "rb")


with open('harry.json', 'w') as fp:
    result = stt.recognize(audio_file, content_type="audio/wav",
                           continuous=True, timestamps=False,
                           max_alternatives=1).get_result()
    json.dump(result, fp, indent=2)
    #print(result)

In [62]:
with open('harry.json') as file:
    distros_dict = json.load(file)
for item in distros_dict['results']:
    print(item['alternatives'][0]['transcript'] +'.')

it is .
stupid go .
Harry potter .
he's dead .
from this day for .
you put your faith .
give me .
wow .
and now is the time to do that yourself .
come forward to join us .
orthodox .
yeah .
JK .
com .
%HESITATION .
well done drink .
well .
well I must say I'd hoped for the best .
and who might you be %HESITATION Matt .
Neville Longbottom .
I will never lie and sure we can find a place for you in all right let's say something .
%HESITATION well novel I'm sure we'd all be fascinated to hear what you have to say .
%HESITATION .
doesn't matter how is gone sundown of people die every day .
friends .
family .
%HESITATION %HESITATION will start tonight .
still with us .
here .
so spread .
Remus .
thanks .
all of .
they didn't die in vain .
you will first the first .


In [63]:
movie=[]
for item in distros_dict['results']:
    movie.append(item['alternatives'][0]['transcript'])
script='. '.join(movie)
script

"it is . stupid go . Harry potter . he's dead . from this day for . you put your faith . give me . wow . and now is the time to do that yourself . come forward to join us . orthodox . yeah . JK . com . %HESITATION . well done drink . well . well I must say I'd hoped for the best . and who might you be %HESITATION Matt . Neville Longbottom . I will never lie and sure we can find a place for you in all right let's say something . %HESITATION well novel I'm sure we'd all be fascinated to hear what you have to say . %HESITATION . doesn't matter how is gone sundown of people die every day . friends . family . %HESITATION %HESITATION will start tonight . still with us . here . so spread . Remus . thanks . all of . they didn't die in vain . you will first the first "

In [64]:
#Preprocessing
import re

processed_movie = re.sub(r'\[[0-9]*\]', ' ', script)  
processed_movie = re.sub(r'\s+', ' ', script)

# Removing special characters and digits
formatted_movie = re.sub('[^a-zA-Z]', ' ', processed_movie )  
formatted_movie = re.sub(r'\s+', ' ', processed_movie)  

In [65]:
#Converting Text To Sentences
import nltk
sentence_list = nltk.sent_tokenize(processed_movie)  
sentence_list

['it is .',
 'stupid go .',
 'Harry potter .',
 "he's dead .",
 'from this day for .',
 'you put your faith .',
 'give me .',
 'wow .',
 'and now is the time to do that yourself .',
 'come forward to join us .',
 'orthodox .',
 'yeah .',
 'JK .',
 'com .',
 '%HESITATION .',
 'well done drink .',
 'well .',
 "well I must say I'd hoped for the best .",
 'and who might you be %HESITATION Matt .',
 'Neville Longbottom .',
 "I will never lie and sure we can find a place for you in all right let's say something .",
 "%HESITATION well novel I'm sure we'd all be fascinated to hear what you have to say .",
 '%HESITATION .',
 "doesn't matter how is gone sundown of people die every day .",
 'friends .',
 'family .',
 '%HESITATION %HESITATION will start tonight .',
 'still with us .',
 'here .',
 'so spread .',
 'Remus .',
 'thanks .',
 'all of .',
 "they didn't die in vain .",
 'you will first the first']

In [74]:
#Find Weighted Frequency of Occurrence

stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}  
for word in nltk.word_tokenize(formatted_movie):  
    if word not in stopwords and word not in "." and word not in "%HESITATION":
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
word_frequencies

{'stupid': 1,
 'go': 1,
 'Harry': 1,
 'potter': 1,
 "'s": 2,
 'dead': 1,
 'day': 2,
 'put': 1,
 'faith': 1,
 'give': 1,
 'wow': 1,
 'time': 1,
 'come': 1,
 'forward': 1,
 'join': 1,
 'us': 2,
 'orthodox': 1,
 'yeah': 1,
 'JK': 1,
 'com': 1,
 'well': 4,
 'done': 1,
 'drink': 1,
 'must': 1,
 'say': 3,
 "'d": 2,
 'hoped': 1,
 'best': 1,
 'might': 1,
 'Matt': 1,
 'Neville': 1,
 'Longbottom': 1,
 'never': 1,
 'lie': 1,
 'sure': 2,
 'find': 1,
 'place': 1,
 'right': 1,
 'let': 1,
 'something': 1,
 'novel': 1,
 "'m": 1,
 'fascinated': 1,
 'hear': 1,
 "n't": 2,
 'matter': 1,
 'gone': 1,
 'sundown': 1,
 'people': 1,
 'die': 2,
 'every': 1,
 'friends': 1,
 'family': 1,
 'start': 1,
 'tonight': 1,
 'still': 1,
 'spread': 1,
 'Remus': 1,
 'thanks': 1,
 'vain': 1,
 'first': 2}

In [75]:
#divide the number of occurances of all the words by the frequency of the most occurring word
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
maximum_frequncy

4

In [76]:
#Calculating Sentence Scores

sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]
sentence_scores

{'stupid go .': 0.5,
 'Harry potter .': 0.25,
 "he's dead .": 0.75,
 'from this day for .': 0.5,
 'you put your faith .': 0.5,
 'give me .': 0.25,
 'wow .': 0.25,
 'and now is the time to do that yourself .': 0.25,
 'come forward to join us .': 1.25,
 'orthodox .': 0.25,
 'yeah .': 0.25,
 'com .': 0.25,
 'well done drink .': 1.5,
 'well .': 1.0,
 "well I must say I'd hoped for the best .": 3.0,
 'and who might you be %HESITATION Matt .': 0.25,
 "I will never lie and sure we can find a place for you in all right let's say something .": 3.5,
 "%HESITATION well novel I'm sure we'd all be fascinated to hear what you have to say .": 3.75,
 "doesn't matter how is gone sundown of people die every day .": 2.75,
 'friends .': 0.25,
 'family .': 0.25,
 '%HESITATION %HESITATION will start tonight .': 0.5,
 'still with us .': 0.75,
 'so spread .': 0.25,
 'thanks .': 0.25,
 "they didn't die in vain .": 1.25,
 'you will first the first': 1.0}

In [78]:
#Getting the Summary
import heapq  

summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)  
print(summary) 

%HESITATION well novel I'm sure we'd all be fascinated to hear what you have to say . I will never lie and sure we can find a place for you in all right let's say something . well I must say I'd hoped for the best . doesn't matter how is gone sundown of people die every day . well done drink .
